In [1]:
import pandas as pd
import numpy as np

# Loading dataset

In [2]:
def read_files(path, filenames):
    """Reads all files and returns a dataframe"""
    return pd.concat((pd.read_csv(path + f, skipinitialspace=True) for f in filenames))

In [3]:
path_opt = "./data/raw_data/"
filenames_opt = ["spx_eod_" + str(year) + (str(month) if month >= 10 else "0"+str(month)) + ".csv" for year in range(2020, 2022) for month in range(1, 13)] + ["spx_eod_2022" + (str(month) if month >= 10 else "0" + str(month)) + ".csv" for month in range(1, 13)]
df = read_files(path_opt, filenames_opt)

# Cleaning raw data

In [4]:
def process_options(df_opt, call = True):
    """Cleans up column names and add time to maturity (TTM)"""
    keys = {key: key[key.find("[")+1:key.find("]")][0] + key[key.find("[")+1:key.find("]")][1:].lower()  for key in df_opt.keys()}
    df_opt = df_opt.rename(columns=keys)

    if call:
        keys = {"C_ask": "Ask", "C_bid": "Bid"}
    else:
        keys = {"P_ask": "Ask", "P_bid": "Bid"}
    df_opt = df_opt.rename(columns=keys)

    df_opt["Quote_date"] = pd.to_datetime(df_opt["Quote_date"])
    df_opt["Expire_date"] = pd.to_datetime(df_opt["Expire_date"])
    df_opt["TTM"] = df_opt.apply(lambda row: (row.Expire_date - row.Quote_date).days, axis = 1)
    df_opt["Price"] = (df_opt["Ask"] + df_opt["Bid"])/2

    columns = ["Quote_date", "Expire_date", "Price", "Underlying_last", "Strike", "TTM"]
    df_opt = df_opt[columns]
    df_opt = df_opt[(df_opt["TTM"] != 0) & (df_opt["TTM"] <= 365*3)]
    return df_opt[columns]

In [5]:
df = process_options(df)

# Adding lags

In [6]:
df['Quote_date']= pd.to_datetime(df['Quote_date'])
df["Moneyness"] = df.apply(lambda row: row.Underlying_last/row.Strike, axis = 1)

In [7]:
# Group the data by Quote Date and calculate the mean for Underlying Price
df_agg = df.groupby('Quote_date').mean().reset_index()

# Values to returns
df_agg["Underlying_return"] = df_agg["Underlying_last"].pct_change()

lags = 90

# Add the Underlying Price Lag column
for i in range(1, lags + 1):
    df_agg['Underlying_' + str(i)] = df_agg['Underlying_return'].shift(i)

df = pd.merge(df, df_agg[['Quote_date', "Underlying_return"] + ['Underlying_' + str(i) for i in range(1, lags + 1)]], on='Quote_date', how='left')

/var/folders/wk/x86_p6511l95p594k6qnb98h0000gn/T/ipykernel_94877/821960197.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_agg = df.groupby('Quote_date').mean().reset_index()


In [8]:
start_date = pd.to_datetime("2021-01-01")
df_val = df[df['Quote_date'] >= start_date]

In [9]:
display(df)

,Quote_date,Expire_date,Price,Underlying_last,Strike,TTM,Moneyness,Underlying_return,Underlying_1,Underlying_2,...,Underlying_81,Underlying_82,Underlying_83,Underlying_84,Underlying_85,Underlying_86,Underlying_87,Underlying_88,Underlying_89,Underlying_90
0,2020-01-02,2020-01-03,1855.705,3258.14,1400.0,1,2.327243,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-02,2020-01-03,1755.645,3258.14,1500.0,1,2.172093,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-02,2020-01-03,1655.955,3258.14,1600.0,1,2.036338,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-02,2020-01-03,1556.195,3258.14,1700.0,1,1.916553,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-02,2020-01-03,1456.210,3258.14,1800.0,1,1.810078,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5758471,2022-12-30,2025-12-19,6.950,3839.81,8400.0,1085,0.457120,-0.0023,0.017343,-0.012034,...,0.017866,-0.003593,-0.000262,-0.010776,0.003305,-0.007857,-0.011077,-0.006737,-0.033678,0.014386
5758472,2022-12-30,2025-12-19,5.950,3839.81,8600.0,1085,0.446490,-0.0023,0.017343,-0.012034,...,0.017866,-0.003593,-0.000262,-0.010776,0.003305,-0.007857,-0.011077,-0.006737,-0.033678,0.014386
5758473,2022-12-30,2025-12-19,5.450,3839.81,8800.0,1085,0.436342,-0.0023,0.017343,-0.012034,...,0.017866,-0.003593,-0.000262,-0.010776,0.003305,-0.007857,-0.011077,-0.006737,-0.033678,0.014386
5758474,2022-12-30,2025-12-19,4.500,3839.81,9000.0,1085,0.426646,-0.0023,0.017343,-0.012034,...,0.017866,-0.003593,-0.000262,-0.010776,0.003305,-0.007857,-0.011077,-0.006737,-0.033678,0.014386


# Matching rates

In [10]:
def process_rates(df_r):
    """Renames rate duration"""
    df_r["Date"] = pd.to_datetime(df_r["Date"])
    keys = {  "Date" : "Quote_date",
                                    "1 Mo": 30,
                                    "3 Mo": 90,
                                    "6 Mo": 180,
                                    "1 Yr": 365,
                                    "2 Yr": 365*2,
                                    "3 Yr": 365*3,
                                    "5 Yr": 365*5,
                                    "7 Yr": 365*7,
                                    "10 Yr": 365*10}
    df_r = df_r.rename(columns = keys)
    return df_r[keys.values()]

def combine_opt_rates(df_opt, df_r):
    df_opt = pd.merge(df_opt, df_r, on ="Quote_date", how = "left")
    rates = list(df_r.columns)
    rates.remove("Quote_date")
    df_opt["TTM_diff"] = df_opt["TTM"].apply(lambda x: (np.abs(np.array(rates) - x)).argmin())
    df_opt["R"] = df_opt[["TTM_diff"] + rates].values.tolist()
    df_opt["R"] = df_opt["R"].apply(lambda x: x[int(x[0]+1)])
    df_opt = df_opt.drop(rates + ["TTM_diff"], axis=1)
    df_opt_len_before = len(df_opt)
    df_opt = df_opt.dropna()
    df_opt_len_after = len(df_opt)
    print("Dropped " + str(df_opt_len_before - df_opt_len_after) + " rows in rate matching")
    return df_opt.dropna()

In [11]:
df_r = pd.concat((pd.read_csv("./data/raw_data/" + f, skipinitialspace=True) for f in ["daily-treasury-rates.csv", "yield-curve-rates-1990-2021.csv"]))
df_r = process_rates(df_r)
df = combine_opt_rates(df, df_r)

Dropped 759742 rows in rate matching


In [ ]:
display(df)

,Quote_date,Expire_date,Price,Underlying_last,Strike,TTM,Moneyness,Underlying_return,Underlying_1,Underlying_2,...,Underlying_82,Underlying_83,Underlying_84,Underlying_85,Underlying_86,Underlying_87,Underlying_88,Underlying_89,Underlying_90,R
704239,2021-05-17,2021-05-19,3062.000,4163.60,1100.0,2,3.785091,-9.88,61.51,48.68,...,-27.55,-14.22,9.03,1.14,-24.63,20.88,55.36,21.16,25.67,0.00
704240,2021-05-17,2021-05-19,2962.505,4163.60,1200.0,2,3.469667,-9.88,61.51,48.68,...,-27.55,-14.22,9.03,1.14,-24.63,20.88,55.36,21.16,25.67,0.00
704241,2021-05-17,2021-05-19,2862.600,4163.60,1300.0,2,3.202769,-9.88,61.51,48.68,...,-27.55,-14.22,9.03,1.14,-24.63,20.88,55.36,21.16,25.67,0.00
704242,2021-05-17,2021-05-19,2761.600,4163.60,1400.0,2,2.974000,-9.88,61.51,48.68,...,-27.55,-14.22,9.03,1.14,-24.63,20.88,55.36,21.16,25.67,0.00
704243,2021-05-17,2021-05-19,2661.600,4163.60,1500.0,2,2.775733,-9.88,61.51,48.68,...,-27.55,-14.22,9.03,1.14,-24.63,20.88,55.36,21.16,25.67,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4059504,2022-12-30,2025-12-19,6.950,3839.81,8400.0,1085,0.457120,-8.85,65.61,-46.08,...,-14.10,-1.03,-42.76,13.07,-31.32,-44.65,-27.34,-141.44,59.56,4.22
4059505,2022-12-30,2025-12-19,5.950,3839.81,8600.0,1085,0.446490,-8.85,65.61,-46.08,...,-14.10,-1.03,-42.76,13.07,-31.32,-44.65,-27.34,-141.44,59.56,4.22
4059506,2022-12-30,2025-12-19,5.450,3839.81,8800.0,1085,0.436342,-8.85,65.61,-46.08,...,-14.10,-1.03,-42.76,13.07,-31.32,-44.65,-27.34,-141.44,59.56,4.22
4059507,2022-12-30,2025-12-19,4.500,3839.81,9000.0,1085,0.426646,-8.85,65.61,-46.08,...,-14.10,-1.03,-42.76,13.07,-31.32,-44.65,-27.34,-141.44,59.56,4.22


## Read to file

In [ ]:
df.to_csv("./data/processed_data/2021_2022.csv")